In [17]:
#r "nuget:Deedle.Interactive,3.0.0-beta.1"
#r "nuget:FSharp.Stats"

// https://stackoverflow.com/a/74260934/977406

Installed Packages Deedle.Interactive, 3.0.0-beta.1 FSharp.Stats, 0.4.8

In [18]:
open Deedle

In [19]:
let df = Frame.ReadCsv("MessyData.csv")

### Wrong data types

In [25]:
// https://stackoverflow.com/a/74260934/977406

let info (df: Deedle.Frame<'a,'b>) =
    let dt = df.GetFrameData()
    let countOptionalValues d =
        d
        |> Seq.filter (
            function
            | OptionalValue.Present _ -> true
            | _ -> false
        )
        |> Seq.length

    Seq.zip dt.ColumnKeys dt.Columns
    |> Seq.map (fun (col, (ty, vec)) ->
        {|
            Column = col
            ``Non-Null Count`` =
                match vec.Data with
                | Vectors.VectorData.DenseList d -> $"%i{d |> Seq.length} non-null"
                | Vectors.VectorData.SparseList d -> $"%i{d |> countOptionalValues} non-null"
                | Vectors.VectorData.Sequence d -> $"%i{d |> countOptionalValues} non-null"
            Dtype = ty
        |}
    )

In [26]:
df |> info

index,Column,Dtype,Non-Null Count
0,[ Name ],System.String,30 non-null
1,[ PhoneNumber ],System.String,30 non-null
2,[ City ],System.String,30 non-null
3,[ Address ],System.String,30 non-null
4,[ PostalCode ],System.String,30 non-null
5,[ BirthDate ],System.DateTime,30 non-null
6,[ Income ],System.Int32,26 non-null
7,[ CreditLimit ],System.String,30 non-null
8,[ MaritalStatus ],System.String,30 non-null


In [ ]:
df |> Frame.fillMissing Direction.Backward

In [108]:
df['CreditLimit'] = pd.to_numeric(df['CreditLimit'], errors='coerce')

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           30 non-null     object 
 1   PhoneNumber    30 non-null     object 
 2   City           30 non-null     object 
 3   Address        30 non-null     object 
 4   PostalCode     30 non-null     object 
 5   BirthDate      30 non-null     object 
 6   Income         26 non-null     float64
 7   CreditLimit    29 non-null     float64
 8   MaritalStatus  24 non-null     object 
dtypes: float64(2), object(7)
memory usage: 2.2+ KB


In [110]:
df['BirthDate'] = pd.to_datetime(df['BirthDate'])

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           30 non-null     object        
 1   PhoneNumber    30 non-null     object        
 2   City           30 non-null     object        
 3   Address        30 non-null     object        
 4   PostalCode     30 non-null     object        
 5   BirthDate      30 non-null     datetime64[ns]
 6   Income         26 non-null     float64       
 7   CreditLimit    29 non-null     float64       
 8   MaritalStatus  24 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.2+ KB


In [112]:
for i in df['BirthDate']:
    print(i.year)

1987
1991
2052
1981
2001
2001
1976
2062
1976
2003
2068
1998
1974
1977
1987
1978
2062
1984
2056
1995
2058
1983
1991
2001
2065
1984
1989
2058
1990
1991


In [140]:
def fix_year(i):
    if i.year > datetime.date.today().year:
        year = i.year - 100
    else:
        year = i.year
    return datetime.datetime(year, i.month, i.day)

In [141]:
df['BirthDate'] = df['BirthDate'].apply(fix_year)

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           30 non-null     object        
 1   PhoneNumber    30 non-null     object        
 2   City           30 non-null     object        
 3   Address        30 non-null     object        
 4   PostalCode     30 non-null     object        
 5   BirthDate      30 non-null     datetime64[ns]
 6   Income         30 non-null     float64       
 7   CreditLimit    30 non-null     float64       
 8   MaritalStatus  30 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.2+ KB


In [115]:
df['PhoneNumber'].unique()

array(['8675550155', '867-555-0148', '613-555-0111', '204-555-0105',
       '902555-0157', '902-555-0177', '204-555-0136', '416-555-0159',
       '204-555-0189', '902-555-0108', '416-5550167', '902-555-0195',
       '204-555-0195', '204-555-0180', '613-555-0154', '613-555-0150',
       '416-5550180', '867-555-0113', '416-555-0192', '902-555-0190',
       '4165550170', '204-555-0190', '902-555-0138', '613-555-0116',
       '613-555-0165', '867-555-0158', '867-555-0161', '416-555-0198',
       '867-555-0134', '613-555-0139'], dtype=object)

In [116]:
phonePattern1 = re.compile(r'^(\d{3})-(\d{3})-(\d{4})$')  #(r'\d\d\d-\d\d\d-\d\d\d\d')
phonePattern2 = re.compile(r'^(\d{10})$')
phonePattern3 = re.compile(r'^(\d{6})-(\d{4})$')
phonePattern4 = re.compile(r'^(\d{3})-(\d{7})$')


def phonePatternFix(x):
    if phonePattern1.search(x) != None:
        return x
    elif phonePattern2.search(x) != None:
        return '-'.join([x[:3],x[3:6],x[6:]])
    elif phonePattern3.search(x) != None:
        return '-'.join([x[:3],x[3:]])
    elif phonePattern4.search(x) != None:
        return '-'.join([x[:7],x[7:]])
    else:
        return "pattern not found"

In [117]:
phonePattern1.search('613-555-0150').group(1)

'613'

In [118]:
df['PhoneNumber'] = df['PhoneNumber'].apply(phonePatternFix)

In [119]:
df['PhoneNumber'].unique()

array(['867-555-0155', '867-555-0148', '613-555-0111', '204-555-0105',
       '902-555-0157', '902-555-0177', '204-555-0136', '416-555-0159',
       '204-555-0189', '902-555-0108', '416-555-0167', '902-555-0195',
       '204-555-0195', '204-555-0180', '613-555-0154', '613-555-0150',
       '416-555-0180', '867-555-0113', '416-555-0192', '902-555-0190',
       '416-555-0170', '204-555-0190', '902-555-0138', '613-555-0116',
       '613-555-0165', '867-555-0158', '867-555-0161', '416-555-0198',
       '867-555-0134', '613-555-0139'], dtype=object)

### Data mistakes

In [120]:
df.City.unique()

array(['Yellowknife', 'Ottawa', 'Winnipeg', 'Charlottetown',
       'Charlotttetownn', 'Toronto', 'Winnipegg', 'Ottawas', 'Toronot'],
      dtype=object)

In [121]:
df['City'] = df['City'].replace(r"^Winnipegg$","Winnipeg", regex=True)
df['City'] = df['City'].replace(r"^Charlotttetownn$","Charlottetown", regex=True)
df['City'] = df['City'].replace(r"^Ottawas$","Ottawa", regex=True)
df['City'] = df['City'].replace(r"^Toronot$","Toronto", regex=True)

In [122]:
df.City.unique()

array(['Yellowknife', 'Ottawa', 'Winnipeg', 'Charlottetown', 'Toronto'],
      dtype=object)

### Missing data

In [123]:
df.dropna().describe(include='all')

,Name,PhoneNumber,City,Address,PostalCode,BirthDate,Income,CreditLimit,MaritalStatus
count,21,21,21,21,21,21,21.000000,21.000000,21
unique,21,21,5,21,19,21,NaN,NaN,3
top,Thane Arthur Andrews,867-555-0155,Yellowknife,25 West Arlington Drive,R3R3B6,1987-01-30,NaN,NaN,Single
freq,1,1,6,1,2,1,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,106222.238095,3147.619048,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,56543.729211,1415.492511,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,18475.000000,500.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,60658.000000,2500.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,117630.000000,3500.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,147312.000000,4000.000000,NaN


In [124]:
df.describe(include='all')

,Name,PhoneNumber,City,Address,PostalCode,BirthDate,Income,CreditLimit,MaritalStatus
count,30,30,30,30,30,30,26.000000,29.000000,24
unique,30,30,5,30,27,30,NaN,NaN,3
top,Thane Arthur Andrews,867-555-0155,Yellowknife,25 West Arlington Drive,R3R3B6,1987-01-30,NaN,NaN,Married
freq,1,1,6,1,2,1,NaN,NaN,9
mean,NaN,NaN,NaN,NaN,NaN,NaN,104742.461538,3055.172414,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,52935.742490,1403.970443,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,18475.000000,500.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,66131.250000,2500.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,107770.500000,3000.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,146980.750000,4000.000000,NaN


In [125]:
df.groupby('MaritalStatus')['Income'].mean()

MaritalStatus
Divorced     93346.000
Married      94709.000
Single      125783.125
Name: Income, dtype: float64

In [126]:
df['MaritalStatus'] = df['MaritalStatus'].fillna('Unknown')

In [127]:
df

,Name,PhoneNumber,City,Address,PostalCode,BirthDate,Income,CreditLimit,MaritalStatus
0,Thane Arthur Andrews,867-555-0155,Yellowknife,25 West Arlington Drive,X1A3C7,1987-01-30,77569.0,500.0,Single
1,Ty Tiina Hume,867-555-0148,Yellowknife,214 West Maple Dr.,X1A3J6,1991-09-05,117630.0,5000.0,Divorced
2,Anita Travert,613-555-0111,Ottawa,24 Constitution St.,K2B8E3,1952-05-02,NaN,3000.0,Married
3,Odilon Vera Ó Maoil Riain,204-555-0105,Winnipeg,7941 Greenview Street,R3G2J6,1981-10-31,97911.0,4000.0,Divorced
4,Dot Cloé Jonasen,902-555-0157,Charlottetown,74 Monroe Street,C1A2V3,2001-07-25,NaN,NaN,Unknown
5,Aoibhín Chinyere Léandre,902-555-0177,Charlottetown,7282 E. Redwood Street,C1A1T1,2001-03-24,157027.0,4500.0,Single
6,Aksel Parent,204-555-0136,Winnipeg,27 Queen Road,R2V1H5,1976-05-19,60658.0,5000.0,Single
7,Helgi Lessie Castelo,416-555-0159,Toronto,438 E. Cypress Ave.,M5V3N2,1962-03-15,NaN,500.0,Divorced
8,Heaven Maximilienne Austin,204-555-0189,Winnipeg,7471 Rockcrest St.,R2G3B5,1976-11-05,152732.0,3500.0,Single
9,Pippa Stéphane Chevalier,902-555-0108,Charlottetown,692 Thompson Drive,C1C1N8,2003-06-06,125318.0,3000.0,Unknown


In [128]:
df['CreditLimit'] = df['CreditLimit'].fillna(df['CreditLimit'].median())

In [129]:
df

,Name,PhoneNumber,City,Address,PostalCode,BirthDate,Income,CreditLimit,MaritalStatus
0,Thane Arthur Andrews,867-555-0155,Yellowknife,25 West Arlington Drive,X1A3C7,1987-01-30,77569.0,500.0,Single
1,Ty Tiina Hume,867-555-0148,Yellowknife,214 West Maple Dr.,X1A3J6,1991-09-05,117630.0,5000.0,Divorced
2,Anita Travert,613-555-0111,Ottawa,24 Constitution St.,K2B8E3,1952-05-02,NaN,3000.0,Married
3,Odilon Vera Ó Maoil Riain,204-555-0105,Winnipeg,7941 Greenview Street,R3G2J6,1981-10-31,97911.0,4000.0,Divorced
4,Dot Cloé Jonasen,902-555-0157,Charlottetown,74 Monroe Street,C1A2V3,2001-07-25,NaN,3000.0,Unknown
5,Aoibhín Chinyere Léandre,902-555-0177,Charlottetown,7282 E. Redwood Street,C1A1T1,2001-03-24,157027.0,4500.0,Single
6,Aksel Parent,204-555-0136,Winnipeg,27 Queen Road,R2V1H5,1976-05-19,60658.0,5000.0,Single
7,Helgi Lessie Castelo,416-555-0159,Toronto,438 E. Cypress Ave.,M5V3N2,1962-03-15,NaN,500.0,Divorced
8,Heaven Maximilienne Austin,204-555-0189,Winnipeg,7471 Rockcrest St.,R2G3B5,1976-11-05,152732.0,3500.0,Single
9,Pippa Stéphane Chevalier,902-555-0108,Charlottetown,692 Thompson Drive,C1C1N8,2003-06-06,125318.0,3000.0,Unknown


In [130]:
df.groupby('City')['Income'].mean()

City
Charlottetown    116642.750000
Ottawa            93529.200000
Toronto          122634.800000
Winnipeg          88424.666667
Yellowknife      107560.833333
Name: Income, dtype: float64

In [131]:
df.groupby('City')['Income'].median()

City
Charlottetown    122042.0
Ottawa            75545.0
Toronto           75039.0
Winnipeg          79284.5
Yellowknife      122699.0
Name: Income, dtype: float64

In [132]:
df['Income'].median()

107770.5

In [133]:
df.groupby('City')['Income'].transform('median')

0     122699.0
1     122699.0
2      75545.0
3      79284.5
4     122042.0
5     122042.0
6      79284.5
7      75039.0
8      79284.5
9     122042.0
10     75039.0
11    122042.0
12     79284.5
13     79284.5
14     75545.0
15     75545.0
16     75039.0
17    122699.0
18     75039.0
19    122042.0
20     75039.0
21     79284.5
22    122042.0
23     75545.0
24     75545.0
25    122699.0
26    122699.0
27     75039.0
28    122699.0
29     75545.0
Name: Income, dtype: float64

In [135]:
df['Income'] = df['Income'].fillna(df.groupby('City')['Income'].transform('median'))

In [136]:
df

,Name,PhoneNumber,City,Address,PostalCode,BirthDate,Income,CreditLimit,MaritalStatus
0,Thane Arthur Andrews,867-555-0155,Yellowknife,25 West Arlington Drive,X1A3C7,1987-01-30,77569.0,500.0,Single
1,Ty Tiina Hume,867-555-0148,Yellowknife,214 West Maple Dr.,X1A3J6,1991-09-05,117630.0,5000.0,Divorced
2,Anita Travert,613-555-0111,Ottawa,24 Constitution St.,K2B8E3,1952-05-02,75545.0,3000.0,Married
3,Odilon Vera Ó Maoil Riain,204-555-0105,Winnipeg,7941 Greenview Street,R3G2J6,1981-10-31,97911.0,4000.0,Divorced
4,Dot Cloé Jonasen,902-555-0157,Charlottetown,74 Monroe Street,C1A2V3,2001-07-25,122042.0,3000.0,Unknown
5,Aoibhín Chinyere Léandre,902-555-0177,Charlottetown,7282 E. Redwood Street,C1A1T1,2001-03-24,157027.0,4500.0,Single
6,Aksel Parent,204-555-0136,Winnipeg,27 Queen Road,R2V1H5,1976-05-19,60658.0,5000.0,Single
7,Helgi Lessie Castelo,416-555-0159,Toronto,438 E. Cypress Ave.,M5V3N2,1962-03-15,75039.0,500.0,Divorced
8,Heaven Maximilienne Austin,204-555-0189,Winnipeg,7471 Rockcrest St.,R2G3B5,1976-11-05,152732.0,3500.0,Single
9,Pippa Stéphane Chevalier,902-555-0108,Charlottetown,692 Thompson Drive,C1C1N8,2003-06-06,125318.0,3000.0,Unknown
